In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import re
import requests
import time
import pandas as pd
import datetime


In [2]:

# These are helpful functions used within the "main functions" 

def find_quater(xxx): 
    if bool(re.search(r'\d{4,4}-[0][1-3]-\d\d', xxx))==True:
        return "QTR1"
    if bool(re.search(r'\d{4,4}-[0][4-6]-\d\d', xxx))==True:
        return "QTR2"
    if bool(re.search(r'\d{4,4}-[0][7-9]-\d\d', xxx))==True:
        return "QTR3"
    if bool(re.search(r'\d{4,4}-[1][0-2]-\d\d', xxx))==True:
        return "QTR4"
    else: 
        print("The date needs to be of the form YYYY-MM-DD.")

    

def add_values_to_dict(sample_dict, type_and_url):
    key = [x[0] for x in type_and_url]
    value_urls = [x[1] for x in type_and_url]
    index = 0
                
    for single_key in key: 
        key_adapted = re.findall(r'\S*\s*\S+', single_key)[0]
        
                    
        if key_adapted not in sample_dict: 
            sample_dict[key_adapted] = list()
        
        sample_dict[key_adapted].append(str(value_urls[index]))
        index = index + 1
      
    return sample_dict





# Disclaimer: one could make the function more pretty by using functions for parts of the code that are 
# repetetive like the functions above. However, I ran out of time and couldn't slim down the code anymore. 


# I tried all variations for edgar_download_htmlfiles that are listed below and they all work (at some point after multiple reruns of the code)
# Further, I included the variable number_of_filings and set it to 11 in order to show that the code does work for
# more than 10 filings. However, even though I accounted for the access regulations of EDGAR by including (too many) time.sleep(0.11)
# in front of every request, I'm still getting access denials somethimes. 
# Therefore if you want to check the code and you receive an error message or no output at all, please rerun the code and maybe set the
# number_of_filing=1 in order to have a better chance to see the code working fully. 


def edgar_download_htmlfiles(date, filetype=None, number_of_filings=11):
    
    #the full index is used because not every date in the daily data has a crawler.date.idx file
    url_main = "https://www.sec.gov/Archives/edgar/full-index/"
    
    
    # here we are looking at cases where the filetype is specified
    # here we dont need to sort by form type, since we are considering only one filetype(=form type)
    if filetype != None: 
    
        if type(date)== str: 
            
            year = str(date)[:4] #this is already a string
            
            #the url used to download the index data of the corresponing date 
            url_crawler = url_main+year+"/"+find_quater(date)+"/crawler.idx"
            time.sleep(0.11) #under 10 requests per second
            crawler_download = requests.get(url_crawler)
            
            time.sleep(0.11)
            soup1 = BeautifulSoup(crawler_download.text, "html.parser")
        
            #get list of all urls that contain the searched filetype and date 
            url_find_htm = re.findall(filetype+r'\s*\d*\s*'+str(date)+r'\s*(\bhttp.*\b)', str(soup1))
             
             
            # interate over the list of urls that have the correct filetype and date
            for wanted in url_find_htm[:number_of_filings]:
                time.sleep(0.11) 
                soup_indiv = BeautifulSoup(requests.get(wanted).text, "html.parser")  
                
                url_find = soup_indiv.table.find_all(string = filetype)
                
                
                # important to use parent here (siblings dont work because find returns Navigatable String and not the <td>) 
                
                # some filings use the same Description as for Type (eg. Description = 8-K, Type=8-K), we are find these Descriptions in our search above
                # therefore we need to account for that here and consider the second match
                if len(url_find) > 1: 
                    url_find_double = url_find[1]
                    td_tag = url_find_double.parent
                    prev_a_tag = td_tag.findPrevious('a')
                else: 
                    td_tag = url_find[0].parent
                    prev_a_tag = td_tag.findPrevious('a')
                
                
                #not all filings are saved in .htm 
                #with this procedure one can download all files of the corresponding type regardless of the file format
                #eg download old .txt filings and new .hml filings
                
                url_file = "https://www.sec.gov"+prev_a_tag['href']
                time.sleep(0.11)
                filing = requests.get(url_file)
                
                file_name = re.search(r'[\w\-]*\.[a-zA-Z]*$', str(prev_a_tag['href']))
                
                
#### DOWNLOADING THE HTML(or other file format) VERSIONS OF THE FILINGS    
                #using the same filename as on the web              
                open(str(file_name.group()), 'wb').write(filing.content)
                
                ########## this is purely used to check if everything worked correctly if the output is <Response [200]> and no other error message occured the entire code execution worked as intended
                print(filing)
                
           
        # here lists of dates are considered with a specific filetype
        if type(date)== list: 
            # use a for loop to interate over all dates. Same as above for one date
            for date_day in date: 
                
                year = date_day[:4] 
                
                 
                url_crawler = url_main+year+"/"+find_quater(date_day)+"/crawler.idx"
                time.sleep(0.11)
                crawler_download = requests.get(url_crawler)
                time.sleep(0.11)
                soup1 = BeautifulSoup(crawler_download.text, "html.parser")
            
                 
                url_find_htm = re.findall(filetype+r'\s*\d*\s*'+str(date_day)+r'\s*(\bhttp.*\b)', str(soup1))
        
                
                for wanted in url_find_htm[:number_of_filings]:
                    time.sleep(0.11) 
                    soup_indiv = BeautifulSoup(requests.get(wanted).text, "html.parser")  
                          
                    
                    url_find = soup_indiv.table.find_all(string = filetype)
                    
                    if len(url_find) > 1: 
                        url_find_double = url_find[1]
                        td_tag = url_find_double.parent
                        prev_a_tag = td_tag.findPrevious('a')
                    else: 
                        td_tag = url_find[0].parent
                        prev_a_tag = td_tag.findPrevious('a')
                    
                    url_file = "https://www.sec.gov"+prev_a_tag['href']
                    time.sleep(0.11)
                    filing = requests.get(url_file)
                     
                    file_name = re.search(r'[\w\-]*\.[a-zA-Z]*$', str(prev_a_tag['href']))
                  
                    open(str(file_name.group()), 'wb').write(filing.content)
                    
                    print(filing)
                
              
    # here we are considering the case where no filetype is specified
    elif filetype == None: 
        # single date
        if type(date)== str: 
            
            year = str(date)[:4] 
            
            url_crawler = url_main+year+"/"+find_quater(date)+"/crawler.idx"
            time.sleep(0.11)
            crawler_download = requests.get(url_crawler)
            time.sleep(0.11)
            soup1 = BeautifulSoup(crawler_download.text, "html.parser")
                
            
            #SINCE MY PC IS SLOW I USED FINDITER INSTEAD OF FINDALL (too many elements found) AND ONLY LOOK AT THE FIRST FEW FINDS
            #THE CODE CAN BE EASILY ADJUSTED TO USE FINDALL (I got on average 10000 matches for any workday, therefore I dont recomend using findall) 
            # the grouping helps getting a tuple of filetype and url
            url_find_htm_with_type = [m.group(1,2) for m, _ in zip(re.finditer(r'\s*(\S+\s?\S*)\s*\d*\s*'+str(date)+r'\s*(\bhttp.*\b)', str(soup1)), range(number_of_filings))]
            
            
# I'm using a dictionary for the filetype (key) and the urls (values)(multiple urls for the same key).
# The dictionary provides the sorted indices. The keys are not in sorted in alphabetical order but after occurance in crawler.idx, if one would like the keys to be ordered alphabetically one could use OrderedDict from collections. 
            url_dict = {}
            add_values_to_dict(url_dict, url_find_htm_with_type)
            
             
            # interate over keys, this is ordered 
            for wanted_type in url_dict.keys():
                
                for wanted in url_dict[str(wanted_type)]:
                    
                    time.sleep(0.11) 
                    soup_indiv = BeautifulSoup(requests.get(wanted).text, "html.parser")  
                              
                    url_find = soup_indiv.table.find_all(string = wanted_type)
                    
                    if len(url_find) > 1: 
                        url_find_double = url_find[1]
                        td_tag = url_find_double.parent
                        prev_a_tag = td_tag.findPrevious('a')
                    else: 
                        td_tag = url_find[0].parent
                        prev_a_tag = td_tag.findPrevious('a')
                    
                    
                    url_file = "https://www.sec.gov"+prev_a_tag['href']
                    time.sleep(0.11)
                    filing = requests.get(url_file)
                    
                   
                    
                    file_name = re.search(r'[\w\-]*\.[a-zA-Z]*$', str(prev_a_tag['href']))
                    
                    open(str(file_name.group()), 'wb').write(filing.content)
                    print(filing)
                
            
        # list of days without filetype    
        if type(date)== list: 
            for date_day in date: 
                                   
                year = date_day[:4] 
                
                url_crawler = url_main+year+"/"+find_quater(date_day)+"/crawler.idx"
                time.sleep(0.11)
                crawler_download = requests.get(url_crawler)
                time.sleep(0.11)
                soup1 = BeautifulSoup(crawler_download.text, "html.parser")
            
                
                url_find_htm_with_type = [m.group(1,2) for m, _ in zip(re.finditer(r'\s*(\S+\s?\S*)\s*\d*\s*'+str(date_day)+r'\s*(\bhttp.*\b)', str(soup1)), range(number_of_filings))]
                
                
                url_dict = {}
                add_values_to_dict(url_dict, url_find_htm_with_type)
                
                             
                
                for wanted_type in url_dict.keys():
                    
                    for wanted in url_dict[str(wanted_type)]:
                        
                        time.sleep(0.11) 
                        soup_indiv = BeautifulSoup(requests.get(wanted).text, "html.parser")  
                                  
                       
                        url_find = soup_indiv.table.find_all(string = wanted_type)
                         
                        if len(url_find) > 1: 
                            url_find_double = url_find[1]
                            td_tag = url_find_double.parent
                            prev_a_tag = td_tag.findPrevious('a')
                        else: 
                            td_tag = url_find[0].parent
                            prev_a_tag = td_tag.findPrevious('a')
                        
                        url_file = "https://www.sec.gov"+prev_a_tag['href']
                        time.sleep(0.11)
                        filing = requests.get(url_file)
                        
                        
                        file_name = re.search(r'[\w\-]*\.[a-zA-Z]*$', str(prev_a_tag['href']))
                        open(str(file_name.group()), 'wb').write(filing.content)
                        print(filing)
                    
                       
                
            
        
    





def edgar_10Q(date, filetype = "10-Q"): 
    # I mainly copied the code from above in the case where we have a specified filetype and one single date
    
    url_main = "https://www.sec.gov/Archives/edgar/full-index/"
    
    year = str(date)[:4]  
    url_crawler = url_main+year+"/"+find_quater(date)+"/crawler.idx"
    time.sleep(0.11)
    crawler_download = requests.get(url_crawler)
    time.sleep(0.11)
    soup1 = BeautifulSoup(crawler_download.text, "html.parser")
        
    url_find_htm = re.findall(filetype+r'\s*\d*\s*'+str(date)+r'\s*(\bhttp.*\b)', str(soup1))
    
        
    for wanted in url_find_htm[:1]:
        
         
        time.sleep(0.11) 
        soup_indiv = BeautifulSoup(requests.get(wanted).text, "html.parser")  
         
        
        url_find = soup_indiv.table.find_all(string = filetype)
                
        if len(url_find) > 1: 
            url_find_double = url_find[1]
            td_tag = url_find_double.parent
            prev_a_tag = td_tag.findPrevious('a')
        else: 
            td_tag = url_find[0].parent
            prev_a_tag = td_tag.findPrevious('a')
                
        url_file = "https://www.sec.gov"+prev_a_tag['href']
        
        time.sleep(0.11)
        filing = requests.get(url_file)
        
# DOWNLOADING THE FILING        
        file_name = re.search(r'[\w\-]*\.[a-zA-Z]*$', str(prev_a_tag['href']))
        open(str(file_name.group()), 'wb').write(filing.content)
        
        
        soup_filing = BeautifulSoup(filing.text, "html.parser")
     
        
# DELETING TABLES AND IMAGES 
        for item in soup_filing.find_all(["table", "picture"]): 
            item.decompose()
        
        
        # use regex to find all items of the form eg. Item 1. Hallo this is not a real item
        # the regex is not ideal since it potentially misses very long "item descriptions". However, it was sufficient for all "item descriptions" I came across 
        item_finder = soup_filing.find_all(string = re.findall(r'[i|I][t|T][e|E][m|M]\s\d+\w*\.\s*\w*\s*\w*\s*\w*\s*\w*\s*\w*\s*\w*\s*\w*\s*\w*\s*\w*', str(soup_filing)))
        
        # create a DataFrame for items and content to be stored
        df = pd.DataFrame([range(len(item_finder))])
        # items are the column names
        df.columns = item_finder
        
        # as before we need to consider the parent and then find the next but one sibling (since the next sibling doesnt contain works in the sting)
        # the content is written in the first row considering the corresponding column
        for i in range(len(item_finder)): 
            tag = item_finder[i].parent
            text_tag = tag.findNext('p').findNext('p')
            df.loc[0][i] = text_tag.string
        
        file_name = re.search(r'[\w\-]*\.[a-zA-Z]*$', str(prev_a_tag['href']))
        open(str(file_name.group()), 'wb').write(filing.content)
        
# RETURNING THE DATAFRAME        
        return df
        
        
        

In [2]:
## all of these function calls worked perfectly at least once, at some point in time
# edgar_download_htmlfiles("2020-02-12", "8-K")
# edgar_download_htmlfiles("2020-02-12")

# date_list = ["2020-02-12", "2019-03-01"]
# edgar_download_htmlfiles(date_list, "8-K")
# edgar_download_htmlfiles(date_list)

I haven't redone the executions to the point, where I don't get an error because I already spent nearly 30 hours on this homework and I'm done. Couldn't figure out where I went wrong with the time.sleep(). Ideally the Response is always 200.
Response 403 or the AttributeError are caused by EDGAR "blocking" me and downloading the document informing me about their policy.

In [104]:
edgar_download_htmlfiles("2020-02-12", "8-K")

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


AttributeError: 'NoneType' object has no attribute 'find_all'

In [86]:
edgar_download_htmlfiles("2020-02-12", None, 1)

<Response [200]>


In [107]:
date_list = ["2020-02-12", "2019-03-01"]
edgar_download_htmlfiles(date_list, "8-K")

<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [17]:
date_list = ["2020-02-12", "2019-03-01"]
edgar_download_htmlfiles(date_list)

<Response [403]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [200]>


In [26]:
## I couldnt execute this function without an error message, typical errors were: 'NoneType' object has no attribute 
## 'find_all' or 'NoneType' object has no attribute 'findNext'
## indicating that at some point in the code execution EDGAR "blocked" me and I downloaded the document informing me 
## about their policy
edgar_10Q("2020-02-11")


AttributeError: 'NoneType' object has no attribute 'findNext'

NOTE: I'll give a brief explanation on how to use a different approch for grabbing the url of the filings: 
One could generate a DataFrame from the table contained in the ...index.html files and than it is fairly easy to extract
the name of the filing from the dataframe. Just search for the type of filing and in the same row one finds the name of the filing document. Next the url needs to be adjusted: when looking at ...index.
I'll demonstrate it with an example: /Archives/edgar/data/1770787/0001193125-20-036029-index.htm is the ...index.htm url
d846921d8k.htm is the filename, and the file url is /Archives/edgar/data/1770787/000119312520036029/d846921d8k.htm
notice here that both urls are nearly the same. One has to simply delete the hyphen (-) from the last part of the url, delete "index.htm" and replace if with "/filename.format"

in retrospect I should have done that instead of grabbing the urls like I did above 